In [0]:
#Assignment(Neural Net)

In [0]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [0]:
# reset graph
tf.reset_default_graph() 

In [0]:
## Loading the data set
raw_data =  load_iris()

In [0]:
## split data set
X_train, X_test, Y_train, Y_test = train_test_split(raw_data.data, raw_data.target, test_size=0.33, random_state=42, stratify= raw_data.target)

In [0]:
## max min scalar on parameters
X_scaler = MinMaxScaler(feature_range=(0,1))

In [0]:
## Preprocessing the dataset
X_train_scaled = X_scaler.fit_transform(X_train)
X_test_scaled = X_scaler.fit_transform(X_test)

In [0]:
## One hot encode Y
onehot_encoder = OneHotEncoder(sparse=False)
Y_train_enc = onehot_encoder.fit_transform(Y_train.reshape(-1,1))
Y_test_enc = onehot_encoder.fit_transform(Y_test.reshape(-1,1))

In [0]:
# Define Model Parameters
learning_rate = 0.03
training_epochs = 8000
 
# define the number of neurons
layer_1_nodes = 1
#layer_2_nodes = 150
 
# define the number of inputs
num_inputs = X_train_scaled.shape[1]
num_output = len(np.unique(Y_train, axis = 0)) 

In [0]:
# Define the layers
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape= (None, num_inputs))
 
with tf.variable_scope('layer_1'):
    weights = tf.get_variable('weights1', shape=[num_inputs, layer_1_nodes], initializer = tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable('bias1', shape=[layer_1_nodes], initializer = tf.zeros_initializer())
    layer_1_output =  tf.nn.relu(tf.matmul(X, weights) +  biases) 
 
#with tf.variable_scope('layer_2'):
#    weights = tf.get_variable('weights2', shape=[layer_1_nodes, layer_2_nodes], initializer = tf.contrib.layers.xavier_initializer())
#    biases = tf.get_variable('bias2', shape=[layer_2_nodes], initializer = tf.zeros_initializer())
#    layer_2_output =  tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)
 
with tf.variable_scope('output'):
    weights = tf.get_variable('weights2', shape=[layer_1_nodes, num_output], initializer = tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable('bias2', shape=[num_output], initializer = tf.zeros_initializer())
    prediction =  tf.matmul(layer_1_output, weights) + biases
 
with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape = (None, num_output))#use 1 instead of num output unless one hot encoding??
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = Y, logits = prediction))
 
with tf.variable_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
 
with tf.variable_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(Y, axis =1), tf.argmax(prediction, axis =1) )
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
 
# Logging results
with tf.variable_scope("logging"):
    tf.summary.scalar('current_cost', cost)
    tf.summary.scalar('current_accuacy', accuracy)
    summary = tf.summary.merge_all()

In [70]:
# Initialize a session so that we can run TensorFlow operations
 
with tf.Session() as session:
 
    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.global_variables_initializer())
 
    # create log file writer to record training progress.
    training_writer = tf.summary.FileWriter(r'C:\data\temp\tf_try\training', session.graph)
    testing_writer = tf.summary.FileWriter(r'C:\data\temp\tf_try\testing', session.graph)
 
    # Run the optimizer over and over to train the network.
    # One epoch is one full run through the training data set.
    for epoch in range(training_epochs):
 
        # Feed in the training data and do one step of neural network training
        session.run(optimizer, feed_dict={X:X_train_scaled, Y:Y_train_enc})
 
        # Every 5 training steps, log our progress
        if epoch %5 == 0:
            training_cost, training_summary = session.run([cost, summary], feed_dict={X: X_train_scaled, Y: Y_train_enc})
            testing_cost, testing_summary = session.run([cost, summary], feed_dict={X: X_test_scaled, Y: Y_test_enc})
 
            #accuracy
            train_accuracy = session.run(accuracy, feed_dict={X: X_train_scaled, Y: Y_train_enc})
            test_accuracy = session.run(accuracy, feed_dict={X: X_test_scaled, Y: Y_test_enc})
 
            print(epoch, training_cost, testing_cost, train_accuracy, test_accuracy )
 
            training_writer.add_summary(training_summary, epoch)
            testing_writer.add_summary(testing_summary, epoch) 
 
    # Training is now complete!
    print("Training is complete!\n")
 
    final_train_accuracy = session.run(accuracy, feed_dict={X: X_train_scaled, Y: Y_train_enc})
    final_test_accuracy = session.run(accuracy, feed_dict={X: X_test_scaled, Y: Y_test_enc})
 
    print("Final Training Accuracy: {}".format(final_train_accuracy))
    print("Final Testing Accuracy: {}".format(final_test_accuracy))
 
    training_writer.close()
    testing_writer.close()

0 1.4190816 1.4283931 0.34 0.32
5 1.3148029 1.3223214 0.34 0.3
10 1.249558 1.2557647 0.34 0.3
15 1.2064552 1.2116464 0.33 0.3
20 1.1765271 1.1809099 0.28 0.24
25 1.1548105 1.1585357 0.15 0.16
30 1.1384263 1.1416003 0.01 0.06
35 1.1256278 1.1283677 0.14 0.2
40 1.11531 1.1177115 0.33 0.34
45 1.1067493 1.1088916 0.33 0.34
50 1.0994563 1.1014057 0.33 0.34
55 1.0930912 1.0949045 0.33 0.34
60 1.087412 1.0891377 0.33 0.34
65 1.0822426 1.083923 0.33 0.34
70 1.0774534 1.0791248 0.33 0.36
75 1.0729463 1.074641 0.34 0.38
80 1.0686467 1.070393 0.34 0.38
85 1.0644969 1.0663192 0.34 0.38
90 1.0604514 1.062372 0.34 0.38
95 1.0564742 1.0585123 0.34 0.38
100 1.0525365 1.0547094 0.34 0.38
105 1.0486146 1.0509378 0.35 0.38
110 1.044689 1.047176 0.35 0.38
115 1.0407436 1.0434066 0.35 0.38
120 1.0367649 1.0396147 0.35 0.38
125 1.0327408 1.0357872 0.35 0.38
130 1.0286615 1.0319132 0.35 0.38
135 1.024518 1.0279828 0.35 0.38
140 1.0203025 1.0239873 0.35 0.4
145 1.016008 1.019919 0.35 0.4
150 1.011628 1.015771